<a href="https://colab.research.google.com/github/anvq38/CS336.L11.KHTN/blob/main/Final_TruyVan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install librosa python_speech_features flask_ngrok flask Flask annoy

In [60]:
import os
import glob
import librosa
from tqdm import tqdm
import numpy as np
from python_speech_features import mfcc, fbank, logfbank
import pickle
from IPython.display import Audio
import pickle
from annoy import AnnoyIndex
import time
from collections import Counter
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename
from flask_ngrok import run_with_ngrok

In [38]:
song = '/content/drive/MyDrive/SongData/Bông Hoa Đẹp Nhất.mp3'
y, sr = librosa.load(song, sr=16000)

In [39]:
print(y)

[ 8.1506022e-07 -3.9518400e-06 -2.4500309e-06 ...  2.0912436e-05
 -1.1530862e-05  0.0000000e+00]


#Difine function

In [61]:
def extract_features(y, sr=16000, nfilt=10, winsteps=0.02):
    try:
        feat = mfcc(y, sr, nfilt=nfilt, winstep=winsteps)
        return feat
    except:
        raise Exception("Extraction feature error")

In [62]:
def crop_feature(feat, i = 0, nb_step=10, maxlen=100):
    crop_feat = np.array(feat[i : i + nb_step]).flatten()
    #print(crop_feat.shape)
    crop_feat = np.pad(crop_feat, (0, maxlen - len(crop_feat)), mode='constant')
    return crop_feat

In [42]:
y, sr = librosa.load('/content/drive/MyDrive/HocKy5/TruyVan/Final_TruyVan/DenDaKhongDuongBeat-AMee-Cut2.mp3', sr=16000)
feat = extract_features(y)
print(feat.shape)
#print(crop_feature(feat))

(4950, 10)


#Load Song

In [43]:
data_dir = '/content/drive/MyDrive/SongData'

In [ ]:
def Load_songs(data_dir):
  features = []
  songs = []
  for song in tqdm(os.listdir(data_dir)):
    if song.endswith('.mp3') or song.endswith('.MP3'):
      song = os.path.join(data_dir, song)
      y, sr = librosa.load(song, sr=16000)
      feat = extract_features(y)
      for i in range(0, feat.shape[0] - 10, 5):
        features.append(crop_feature(feat, i, nb_step=10))
        songs.append(song[32:])
  return features, songs

In [ ]:
Features, Songs = Load_songs(data_dir)

100%|██████████| 1070/1070 [4:13:11<00:00, 14.20s/it]


##Save Features, Songs to drive

In [ ]:
pickle.dump(Features, open('/content/drive/MyDrive/HocKy5/TruyVan/Final_TruyVan/features.pk', 'wb'))

pickle.dump(Songs, open('/content/drive/MyDrive/HocKy5/TruyVan/Final_TruyVan/songs.pk', 'wb'))

# Load: Feature, Songs from drive

In [63]:
Features = pickle.load(open('/content/drive/MyDrive/HocKy5/TruyVan/Final_TruyVan/features.pk', 'rb'))
Songs = pickle.load(open('/content/drive/MyDrive/HocKy5/TruyVan/Final_TruyVan/songs.pk', 'rb'))

In [64]:
print(len(Features))

2722733


#Truy vấn bằng Annoy

##Add dữ liệu vào trong annoy indexing

In [ ]:
#add dữ liệu vào trong annoy indexing

f = 100
t = AnnoyIndex(f)

for i in range(1359671):
    v = Features[i]
    t.add_item(i, v)
  
f_ = 100
t_ = AnnoyIndex(f_)

for i in range(1359671, len(Features)):
    v = Features[i]
    t_.add_item(i, v)


##Tạo cây index

In [ ]:
t.build(100) # 100 trees
t.save('/content/drive/MyDrive/HocKy5/TruyVan/Final_TruyVan/Music.ann')

t_.build(100)
t_.save('/content/drive/MyDrive/HocKy5/TruyVan/Final_TruyVan/Music_.ann')

True

#Load trees from drive

In [ ]:
#Load
f, f_ = 100, 100
u = AnnoyIndex(f)
u.load('/content/drive/MyDrive/HocKy5/TruyVan/Final_TruyVan/Music.ann')

u_ = AnnoyIndex(f_)
u_.load('/content/drive/MyDrive/HocKy5/TruyVan/Final_TruyVan/Music_.ann')

#Demo

In [66]:
def Demo(song):
  start = time.time()
  y, sr = librosa.load(song, sr = 16000)
  feat = extract_features(y)
  print('Thoi gian load file am thanh : ', time.time() - start)
  results, ans = [], []
  for i in range(0, feat.shape[0], 10):
      crop_feat = crop_feature(feat, i, nb_step=10)
      result = u.get_nns_by_vector(crop_feat, n=5)
      result_songs = [Songs[k] for k in result]
      results.append(result_songs)

  for i in range(0, feat.shape[0], 10):
      crop_feat = crop_feature(feat, i, nb_step=10)
      result = u_.get_nns_by_vector(crop_feat, n=5)
      result_songs = [Songs[k] for k in result]
      results.append(result_songs)
      
  results = np.array(results).flatten()
  most_song = Counter(results)
  A = np.array(most_song.most_common()[:5])
  Time = time.time() - start
  print('Thoi gian truy van : ', Time)
  for i in range(len(A)):
    ans.append(A[i][0])
  return ans
  #print(np.array(ans).shape)

In [67]:
song = input('Path to song: ')
print(Demo(song))

Path to song: /content/drive/MyDrive/HocKy5/TruyVan/Final_TruyVan/DenDaKhongDuongBeat-AMee-Cut1.mp3
Thoi gian load file am thanh :  6.263116359710693
Thoi gian truy van :  70.6675922870636
['Đen Đá Không Đường.mp3', 'Mãi Mãi Là Một Lời Nói Dối (Piano Version).mp3', 'This City.mp3', 'Star And Moon.mp3', 'Hallelujah.mp3']


#Flask

In [59]:
# from app import app
# import urllib.request

path_f = '/content/static/music'

ALLOWED_EXTENSIONS = set(['wav','mp3'])


UPLOAD_FOLDER = 'static/music/'

app = Flask(__name__)
run_with_ngrok(app)
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024


def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
	
@app.route('/')
def upload_form():
	return render_template('index2.html')

@app.route('/', methods=['POST'])
def upload_image():
	if 'file' not in request.files:
		flash('No file part')
		return redirect(request.url)
	file = request.files['file']
	if file.filename == '':
		flash('No image selected for uploading')
		return redirect(request.url)
	if file and allowed_file(file.filename):
		filename = secure_filename(file.filename)
		file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
		#print('upload_image filename: ' + filename)
		flash('successfully uploaded')
		result = Demo(os.path.join(path_f, filename))
		# , result0=result[0],result1=result[1],result2=result[2],result3=result[3],result4=result[4]
		return render_template('index2.html', filename=filename, result= result)
	else:
		flash('Allowed image types are -> wav,mp3')
		return redirect(request.url)

@app.route('/display/<filename>')
def display_image(filename):
	#print('display_image filename: ' + filename)
	return redirect(url_for('static', filename='music/' + filename), code=301)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a23477c9ab7a.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [17/Jan/2021 14:27:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2021 14:27:12] "GET /favicon.ico HTTP/1.1" 404 -


Thoi gian load file am thanh :  4.8750762939453125


127.0.0.1 - - [17/Jan/2021 14:27:28] "POST / HTTP/1.1" 200 -


Thoi gian truy van :  5.212013483047485


127.0.0.1 - - [17/Jan/2021 14:27:28] "GET /display/DenDaKhongDuongBeat-AMee-Cut1.mp3 HTTP/1.1" 301 -


In [70]:
pip freeze > requirements.txt